In [6]:
import pandas as pd
import folium
import numpy as np
import json
import zipfile
import os
import geopandas as gpd
import time
import datetime
import itertools
import requests
import bs4
from folium.plugins.timedynamic_geo_json import TimeDynamicGeoJson
from branca.colormap import linear
import branca.colormap

In [7]:
pd.options.mode.chained_assignment = None

In [8]:
with open('data/countries.txt', 'r') as fp:
    countries = [f.strip('\n') for f in fp.readlines()]

## READ JSON DATA

In [9]:
def json_reader(path):
    with open(path, 'r', encoding='utf-8') as fh:
        return json.loads(fh.read())

In [10]:
geojsons = {c: json_reader(f"data/{c.capitalize()}_AL4.GeoJson") for c in countries}

## GEOPANDAS FOR ADDITIONAL DATA

In [12]:
# concatenate data with country flags
for c in countries:
    gdf_ = gpd.GeoDataFrame.from_features(geojsons[c])
    gdf_['country'] = c
    try:
        gdf = gdf.append(gdf_, sort = False)
    except:
        gdf = gdf_

# drop some columns and calculate centroids
gdf = gdf[['geometry', 'name', 'localname', 'country']]
gdf['centr_lat'] = gdf.geometry.centroid.x
gdf['centr_lon'] = gdf.geometry.centroid.y
gdf = gdf.dropna()
gdf = gdf.drop_duplicates(subset=['name'])
gdf.head()

,geometry,name,localname,country,centr_lat,centr_lon
0,"(POLYGON ((104.4190919 15.6978084, 104.4190978...",Amnat Charoen Province,จังหวัดอำนาจเจริญ,thailand,104.742211,15.896505
1,"(POLYGON ((100.1949768 14.5658321, 100.1960907...",Ang Thong Province,จังหวัดอ่างทอง,thailand,100.355477,14.623284
2,"(POLYGON ((100.3278772 13.8041844, 100.3318503...",Bangkok,กรุงเทพมหานคร,thailand,100.623651,13.771980
3,"(POLYGON ((103.24334 18.357593, 103.2434 18.35...",Bueng Kan Province,จังหวัดบึงกาฬ,thailand,103.712064,18.148721
4,"(POLYGON ((102.4355316 14.8494415, 102.4369202...",Buri Ram Province,จังหวัดบุรีรัมย์,thailand,102.962893,14.815251


In [13]:
gdf.shape

(136, 6)

## CREATE AREAS WE HAVE DATA ON

In [14]:
## THAILAND
bangkok = ['Prachuap Khiri Khan Province', 'Phetchaburi Province', 'Ratchaburi Province', 'Samut Songkhram Province',
           'Nakhon Pathom Province', 'Samut Sakhon Province', 'Nonthaburi Province', 'Pathum Thani Province',
           'Bangkok', 'Samut Prakan Province', 'Nakhon Nayok Province', 'Prachin Buri Province', 'Sa Kaeo Province', 
           'Chachoengsao Province', 'Chon Buri Province']

rainycoast = ['Rayong Province', 'Chanthaburi Province', 'Trat Province']

peni_east = ['Prachuap Khiri Khan Province', 'Chumphon Province', 'Surat Thani Province', 'Nakhon Si Thammarat Province',
             'Phatthalung Province', 'Songkhla Province', 'Pattani Province', 'Yala Province', 'Narathiwat Province']

peni_west = ['Ranong Province', 'Phangnga Province', 'Phuket Province', 'Krabi Province', 'Trang Province',
             'Satun Province']

mountains = ['Tak Province', 'Mae Hong Son Province', 'Chiang Mai Province', 'Chiang Rai Province', 'Phayao Province',
             'Nan Province', 'Uttaradit Province', 'Phitsanulok Province', 'Loei Province']

all_oth_thai = bangkok + rainycoast + peni_east + peni_west + mountains
north = [f for f in gdf[gdf['country'] == 'thailand']['name'].tolist() if not f in all_oth_thai]

div_map = {f: 'Bangkok' for f in bangkok}
div_map.update({f: 'Rainycoast' for f in rainycoast})
div_map.update({f: 'Eastern peninsular' for f in peni_east})
div_map.update({f: 'Wester peninsular' for f in peni_west})
div_map.update({f: 'Mountains' for f in mountains})
div_map.update({f: 'North' for f in north})

In [15]:
## CAMBODIA
coast = ['Koh Kong', 'Preah Sihanouk', 'Kampot', 'Kep']
inside = [f for f in gdf[gdf['country'] == 'cambodia']['name'].tolist() if not f in coast]

div_map.update({f: 'Coastal Cambodia' for f in coast})
div_map.update({f: 'Inside Cambodia' for f in inside})

In [16]:
gdf['regions'] = gdf['name'].map(div_map)
gdf.head()

,geometry,name,localname,country,centr_lat,centr_lon,regions
0,"(POLYGON ((104.4190919 15.6978084, 104.4190978...",Amnat Charoen Province,จังหวัดอำนาจเจริญ,thailand,104.742211,15.896505,North
1,"(POLYGON ((100.1949768 14.5658321, 100.1960907...",Ang Thong Province,จังหวัดอ่างทอง,thailand,100.355477,14.623284,North
2,"(POLYGON ((100.3278772 13.8041844, 100.3318503...",Bangkok,กรุงเทพมหานคร,thailand,100.623651,13.771980,Bangkok
3,"(POLYGON ((103.24334 18.357593, 103.2434 18.35...",Bueng Kan Province,จังหวัดบึงกาฬ,thailand,103.712064,18.148721,North
4,"(POLYGON ((102.4355316 14.8494415, 102.4369202...",Buri Ram Province,จังหวัดบุรีรัมย์,thailand,102.962893,14.815251,North


In [17]:
print(gdf.shape)
gdf = gdf.dropna()
gdf.shape

(136, 7)


(102, 7)

In [18]:
gdf_dis = gdf.dissolve(by='regions')

In [19]:
gdf_dis = gdf_dis.drop('name', axis = 1).reset_index()
gdf_dis['centr_lat'] = gdf_dis.geometry.centroid.x
gdf_dis['centr_lon'] = gdf_dis.geometry.centroid.y

In [20]:
gdf_dis

,regions,geometry,localname,country,centr_lat,centr_lon
0,Bangkok,"(POLYGON ((100.972446 12.5163367, 100.9722184 ...",กรุงเทพมหานคร,thailand,100.893977,13.606975
1,Coastal Cambodia,"(POLYGON ((102.8802919 9.9327889, 102.8810376 ...",ខេត្តកំពត,cambodia,103.703957,11.230224
2,Eastern peninsular,"(POLYGON ((100.7437142 8.375142800000001, 100....",จังหวัดชุมพร,thailand,99.964067,8.481450
3,Inside Cambodia,"POLYGON ((104.4101661 11.110994, 104.4102828 1...",ខេត្តបន្ទាយមានជ័យ,cambodia,105.052908,12.877208
4,Mountains,"POLYGON ((97.9100251 17.5648335, 97.9105157 17...",จังหวัดเชียงใหม่,thailand,99.660011,18.245166
5,North,"POLYGON ((105.0600095 16.0978688, 105.064462 1...",จังหวัดอำนาจเจริญ,thailand,101.903237,16.037033
6,Rainycoast,"(POLYGON ((102.3384216 12.187277, 102.3386017 ...",จังหวัดจันทบุรี,thailand,102.012957,12.735317
7,Wester peninsular,"(POLYGON ((99.7511875 6.7710485, 99.7520313 6....",จังหวัดกระบี่,thailand,99.086736,8.230258


In [51]:
geojson = json.loads(gdf_dis.to_json())

In [53]:
len(geojson['features'])

8

## GET WEATHER DATA

https://www.climatestotravel.com/climate/thailand

In [32]:
def get_country_weather(country):
    soup = soup_country_page(country)
    captions, dfs = scrape_tables(soup)
    return parse_tables_captions(captions, dfs)

def soup_country_page(country):
    r = requests.get(f'https://www.climatestotravel.com/climate/{country}')
    return bs4.BeautifulSoup(r.content, 'html5lib')

def scrape_tables(soup):
    tables = soup.find_all('table')
    dfs = []
    captions = []
    for table in tables:
        caption = table.find_all('caption')
        table_rows = table.find_all('tr')
        res = []
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text.strip() for tr in td if tr.text.strip()]
            if row:
                res.append(row)
        df = pd.DataFrame(res)
        dfs.append(df)
        captions.append(caption[0].text)
    
    assert len(captions) == len(dfs), 'Not matching number of captions and tables!'
    return captions, dfs

def parse_tables_captions(captions, dfs):
    new_dfs = {}
    for cap, df in zip(captions, dfs):
        if (('temper' in cap) & ('Sea' not in cap)):
            df.index = ['MinC', 'MaxC',  'MinF', 'MaxF']
            df.columns = [f for f in range(1,13)]
        elif 'precip' in cap:
            df.index = ['Prec(mm)', 'Prec(in)', 'Prec(days)']
            df.columns = [f for f in range(1,13)] + ['Year']
        elif 'Sunshin' in cap:
            df.index = ['HoursDaily']
            df.columns = [f for f in range(1,13)]
        else:
            continue
        new_dfs[cap] = df
    return new_dfs

In [33]:
weathers = {c: get_country_weather(c) for c in countries}

## MUNGE WEATHER DATA

In [34]:
# now only 2 countries
countries = ['thailand', 'cambodia']

weathers = {k: v for k, v in weathers.items() if k in countries}

In [35]:
weather_to_region = {'Chiang Mai': 'North', 'Bangkok': 'Bangkok', 'Pattaya': None, 
                     'Ko Samui': 'Eastern peninsular', 'Phuket': 'Western Peninsular',
                     'Phnom Penh': 'Inside Cambodia', 'Kampot': 'Coastal Cambodia'}

In [36]:
def translate_weather_to_region(weather_annot, weather_to_region):
    return weather_to_region[weather_annot.split("-")[0].strip()]

In [37]:
weather_data = {r: {} for r in weather_to_region.values()}
for c, data in weathers.items():
    for key, val in data.items():
        region = translate_weather_to_region(key, weather_to_region)
        if region is None:
                continue
        for col in val.columns:
            val[col] = val[col].astype(float)
        if 'temper' in key:
            val = val.iloc[:2]
            val = pd.DataFrame(val.mean()).T
            val.index = [region]
            try:
                weather_data['temper'] = weather_data['temper'].append(val)
            except:
                weather_data['temper'] = val
            weather_data[region]['temper'] = val
        if 'precip' in key:
            val = pd.DataFrame(val.iloc[0].drop('Year')).T
            val.index = [region]
            try:
                weather_data['precip'] = weather_data['precip'].append(val)
            except:
                weather_data['precip'] = val

In [38]:
weather_data['temper'].dropna()

,1,2,3,4,5,6,7,8,9,10,11,12
North,21.5,24.0,27.0,29.0,29.0,28.5,28.0,27.5,27.5,26.5,24.5,21.5
Bangkok,26.5,28.0,29.5,30.5,30.0,29.0,29.0,29.0,28.5,28.0,27.5,26.0
Eastern peninsular,26.5,27.5,28.5,29.0,29.5,29.0,28.5,28.5,28.5,27.5,27.0,26.5
Western Peninsular,27.0,27.5,28.5,29.0,28.0,28.0,28.0,27.5,27.0,27.5,27.5,27.5
Inside Cambodia,27.0,28.0,29.5,30.0,29.5,29.5,29.0,29.0,28.0,27.5,26.5,26.0
Coastal Cambodia,26.0,26.5,27.5,28.5,28.5,27.5,27.0,27.0,26.5,27.0,26.5,26.0


## CREATE COLOR CODING

In [90]:
def get_color(cmap, r, month, type_, weather_data=weather_data, gdf_dis=gdf_dis,):
    try:
        return cmap(weather_data[type_].loc[gdf_dis.loc[r,'regions'], month])
    except:
        return "000000"

In [91]:
min_temp = weather_data['temper'].min().min()
max_temp = weather_data['temper'].max().max()
min_precip = weather_data['precip'].min().min()
max_precip = weather_data['precip'].max().max()

In [92]:
cmap_temper = branca.colormap.LinearColormap(colors = ('yellow', 'red'), vmin = min_temp, vmax = max_temp)
cmap_precip = branca.colormap.LinearColormap(colors = ('white', 'blue'), vmin = min_precip, vmax = max_precip)

In [95]:
# opacity will be standard
opa = 0.5

styledata_temper = {r: 
             {f'2018-{str(month)}-1':
             {'color': get_color(cmap_temper, r, month, 'temper'),
              'opacity': opa}
              for month in range(1, 13)}
             for r in range(8)}

styledata_precip = {r: 
             {f'2018-{str(month)}-1':
             {'color': get_color(cmap_precip, r, month, 'precip'),
              'opacity': opa}
              for month in range(1, 13)}
             for r in range(8)} 


## CREATE MAP

In [100]:
m = folium.Map(location = (15.896505, 104.742211), zoom_start=8)

In [102]:
m_temper = folium.Map(location = (15.896505, 104.742211), zoom_start=8)

g = TimeDynamicGeoJson(
    gdf_dis.to_json(),
    styledict = styledata_temper,
    
).add_to(m_temper)

m_precip = folium.Map(location = (15.896505, 104.742211), zoom_start=8)

g2 = TimeDynamicGeoJson(
    gdf_dis.to_json(),
    styledict = styledata_precip,
    
).add_to(m_precip)

m_precip.save('outputs/precipitation_timeslider.html')
m_temper.save('outputs/temperature_timeslider.html')

In [31]:
for k, v in geojsons.items():
    folium.GeoJson(
        v,
        name=k.capitalize()
    ).add_to(m)

fg = folium.FeatureGroup(name='Regions')
for name, lat, lon in zip(gdf['name'].tolist(), gdf['centr_lat'].tolist(), gdf['centr_lon'].tolist()):
    fg.add_child(folium.Marker(location=[lon, lat], popup=name))
    
m.add_child(fg)

folium.LayerControl().add_to(m)

In [32]:
m.save('outputs/map.html')